In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
races = {
        'season': [],
        'round': [],
        'circuit_id': [],
        'country': [],
        'lat': [],
        'long': [],
        'date': [],
        'url': []
        }

years = [2020, 2021]

In [3]:
for year in years:
    url = 'https://ergast.com/api/f1/{}.json'
    r = requests.get(url.format(year))
    json = r.json()

    for item in json['MRData']['RaceTable']['Races']:
        races['season'].append(int(item['season']))
        races['round'].append(int(item['round']))
        races['circuit_id'].append(item['Circuit']['circuitId'])
        races['country'].append(item['Circuit']['Location']['country'])    
        races['lat'].append(float(item['Circuit']['Location']['lat']))
        races['long'].append(float(item['Circuit']['Location']['long']))
        races['date'].append(item['date'])
        races['url'].append(item['url'])

races = pd.DataFrame(races)

races.head()

,season,round,circuit_id,country,lat,long,date,url
0,2020,1,red_bull_ring,Austria,47.2197,14.76470,2020-07-05,http://en.wikipedia.org/wiki/2020_Austrian_Gra...
1,2020,2,red_bull_ring,Austria,47.2197,14.76470,2020-07-12,http://en.wikipedia.org/wiki/2020_Styrian_Gran...
2,2020,3,hungaroring,Hungary,47.5789,19.24860,2020-07-19,http://en.wikipedia.org/wiki/2020_Hungarian_Gr...
3,2020,4,silverstone,UK,52.0786,-1.01694,2020-08-02,http://en.wikipedia.org/wiki/2020_British_Gran...
4,2020,5,silverstone,UK,52.0786,-1.01694,2020-08-09,http://en.wikipedia.org/wiki/70th_Anniversary_...


In [4]:
races.to_csv('races.csv')

In [5]:
race = pd.read_csv('races.csv')

rounds = []
for year in np.array(race.season.unique()):
    rounds.append([year, list(race[race.season == year]['round'])])

# query API
    
results = {
    'season': [],
    'round':[],
    'circuit_id':[],
    'driver': [],
    'date_of_birth': [],
    'nationality': [],
    'constructor': [],
    'grid': [],
    'time': [],
    'status': [],
    'points': [],
    'podium': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'http://ergast.com/api/f1/{}/{}/results.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['RaceTable']['Races'][0]['Results']:

            results['season'].append(int(json['MRData']['RaceTable']['Races'][0]['season']))
            results['round'].append(int(json['MRData']['RaceTable']['Races'][0]['round']))
            results['circuit_id'].append(json['MRData']['RaceTable']['Races'][0]['Circuit']['circuitId'])
            results['driver'].append(item['Driver']['driverId'])
            results['date_of_birth'].append(item['Driver']['dateOfBirth'])
            results['nationality'].append(item['Driver']['nationality'])
            results['constructor'].append(item['Constructor']['constructorId'])
            results['grid'].append(int(item['grid']))

            try:
                results['time'].append(int(item['Time']['millis']))
            except:
                results['time'].append(None)

            results['status'].append(item['status'])
            results['points'].append(float(item['points']))
            results['podium'].append(int(item['position']))


results = pd.DataFrame(results)
results.to_csv('results.csv', index = False)

In [6]:
results.query('season == 2020')['driver'].unique()

array(['bottas', 'leclerc', 'norris', 'hamilton', 'sainz', 'perez',
       'gasly', 'ocon', 'giovinazzi', 'vettel', 'latifi', 'kvyat',
       'albon', 'raikkonen', 'russell', 'grosjean', 'kevin_magnussen',
       'stroll', 'ricciardo', 'max_verstappen', 'hulkenberg', 'aitken',
       'pietro_fittipaldi'], dtype=object)

In [7]:
driver_standings = {
    'season': [],
    'round':[],
    'driver': [],
    'driver_points': [],
    'driver_wins': [],
    'driver_standings_pos': []}

for n in list(range(len(rounds))):
    for i in rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/driverStandings.json'
        r = requests.get(url.format(rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']:
            driver_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            driver_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))    
            driver_standings['driver'].append(item['Driver']['driverId'])
            driver_standings['driver_points'].append(float(item['points']))
            driver_standings['driver_wins'].append(int(item['wins']))
            driver_standings['driver_standings_pos'].append(int(item['position']))

driver_standings = pd.DataFrame(driver_standings)
driver_standings.to_csv('driver_standings.csv', index = False)
driver_standings.tail(21)

,season,round,driver,driver_points,driver_wins,driver_standings_pos
787,2021,22,max_verstappen,395.5,10,1
788,2021,22,hamilton,387.5,8,2
789,2021,22,bottas,226.0,1,3
790,2021,22,perez,190.0,1,4
791,2021,22,sainz,164.5,0,5
792,2021,22,norris,160.0,0,6
793,2021,22,leclerc,159.0,0,7
794,2021,22,ricciardo,115.0,1,8
795,2021,22,gasly,110.0,0,9
796,2021,22,alonso,81.0,0,10


In [8]:
constructor_rounds = rounds

constructor_standings = {'season': [],
                    'round':[],
                    'constructor': [],
                    'constructor_points': [],
                    'constructor_wins': [],
                   'constructor_standings_pos': []}

for n in list(range(len(constructor_rounds))):
    for i in constructor_rounds[n][1]:
    
        url = 'https://ergast.com/api/f1/{}/{}/constructorStandings.json'
        r = requests.get(url.format(constructor_rounds[n][0], i))
        json = r.json()

        for item in json['MRData']['StandingsTable']['StandingsLists'][0]['ConstructorStandings']:
            constructor_standings['season'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['season']))
            constructor_standings['round'].append(int(json['MRData']['StandingsTable']['StandingsLists'][0]['round']))
            constructor_standings['constructor'].append(item['Constructor']['constructorId'])
            constructor_standings['constructor_points'].append(float(item['points']))
            constructor_standings['constructor_wins'].append(int(item['wins']))
            constructor_standings['constructor_standings_pos'].append(int(item['position']))
            
constructor_standings = pd.DataFrame(constructor_standings)
constructor_standings.to_csv('constructor_standings.csv', index = False)
constructor_standings.tail(10)

,season,round,constructor,constructor_points,constructor_wins,constructor_standings_pos
380,2021,22,mercedes,613.5,9,1
381,2021,22,red_bull,585.5,11,2
382,2021,22,ferrari,323.5,0,3
383,2021,22,mclaren,275.0,1,4
384,2021,22,alpine,155.0,1,5
385,2021,22,alphatauri,142.0,0,6
386,2021,22,aston_martin,77.0,0,7
387,2021,22,williams,23.0,0,8
388,2021,22,alfa,13.0,0,9
389,2021,22,haas,0.0,0,10
